In [196]:
%load_ext autoreload

%autoreload 2

from glob import glob
from os.path import join
import os
import pickle
import random

import scipy.linalg
import scipy.stats
import matplotlib
matplotlib.rcParams["figure.dpi"] = 144
matplotlib.rcParams["savefig.dpi"] = 144
import matplotlib.pyplot as plt
import numpy as np

import subprocess
import shutil

from astropy.table import Table, Column
import astropy.units as u

import emcee

import kepler
import threebody

import minuit_wrap
%matplotlib inline

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Check everything is ready

In [120]:
!ls -l fitter_params.pickle emcee_params.pickle

-rw-r--r-- 1 aarchiba aipsuser 513 Aug  8 17:13 emcee_params.pickle
-rw-r--r-- 1 aarchiba aipsuser 439 Aug 15 10:21 fitter_params.pickle


In [113]:
shutil.copy("fitter_params.pickle","emcee_params.pickle")

In [339]:
shutil.copy("downhill-runs/WSRT/fitter_params.pickle","emcee_params.pickle")

In [340]:
d = pickle.load(open("emcee_params.pickle"))
F = threebody.Fitter(**d)
d

{'dmx_span': (144.3, 509.55, 874.8, 1240.05, 1605.3, 1970.55),
 'efac': 1,
 'fdn_range': (1, 5),
 'files': 'tuned2_aligned_pulses_5deg_WSRT',
 'fit_pm': True,
 'fit_pos': True,
 'fit_px': True,
 'kopeikin': False,
 'linear_dm': False,
 'linear_jumps': True,
 'parfile': '0337_de435.par',
 'ppn_mode': 'heavysimple',
 'priors': ('dbeta', 'dgamma'),
 't2_astrometry': True,
 'toa_mode': 'pipeline',
 'tzrmjd_middle': 'auto',
 'variable_dm': True,
 'variable_ipm': True}

In [341]:
F.goodness_of_fit()

4100.7956183974937669

## Set up walkers and cores

In [328]:
matching_run = None
runs = sorted((os.path.getmtime(g), g) for g in glob("emcee-chains/*"))
for r in runs[::-1]:
    p = join(r[1],"parameters.npy")
    if not os.path.exists(p): 
        continue
    p = list(np.load(p))
    if p == F.parameters:
        matching_run = r[1]
        break
else:
    raise ValueError("No matching set of parameters found")
print matching_run

emcee-chains/5555.nimrod.cv.nrao.edu


In [329]:
setup = "new"

if setup == "new":
    nodes = 5
    ppn = 20
    cycles = 1
    tag = "new"
elif setup == "old":
    nodes = 7
    ppn = 12
    cycles = 1
    tag = "old"
    
walkers = nodes*ppn*cycles-1
walkers -= walkers % 2
walkers

98

In [330]:
skip = 20

input_walkers = sorted(glob(join(matching_run,"*-pos.npy")))[::-skip]

new_walker_pool = []
for w in input_walkers:
    nw = np.load(w)
    if not np.all(np.isfinite(nw)):
        continue
    if len(nw.shape) != 2:
        raise ValueError("Bad shape for walkers: %s" % (nw.shape,))
    nw -= nw.mean(axis=0)[None,:]
    new_walker_pool.extend(list(nw))

if len(new_walker_pool)<walkers:
    raise ValueError("Only %d walkers found but needed %d" 
                     % (len(new_walker_pool), walkers))

random.shuffle(new_walker_pool)
new_walkers = np.array(new_walker_pool[:walkers])[None,:,:]

In [331]:
walker_name = "start-walkers.npy"
# Note shape has a 1 in front
np.save(walker_name, new_walkers)
print new_walkers.shape

(1, 98, 17)


## No matching version

In [288]:
matching_run = None
runs = sorted((os.path.getmtime(g), g) for g in glob("emcee-chains/*"))
for r in runs[::-1]:
    p = join(r[1],"parameters.npy")
    if not os.path.exists(p): 
        continue
    p = list(np.load(p))
    if set(p) >= set(F.parameters):
        matching_run = r[1]
        ix = [p.index(pi) for pi in F.parameters]
        break
else:
    raise ValueError("No matching set of parameters found")
print matching_run
print ix
print p
print F.parameters

emcee-chains/4391.nimrod.cv.nrao.edu
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
['asini_i', 'pb_i', 'eps1_i', 'eps2_i', 'tasc_i', 'acosi_i', 'q_i', 'asini_o', 'pb_o', 'eps1_o', 'eps2_o', 'tasc_o', 'acosi_o', 'delta_lan', 'delta', 'dgamma', 'dbeta']
['asini_i', 'pb_i', 'eps1_i', 'eps2_i', 'tasc_i', 'acosi_i', 'q_i', 'asini_o', 'pb_o', 'eps1_o', 'eps2_o', 'tasc_o', 'acosi_o', 'delta_lan', 'delta']


In [289]:
skip = 20

input_walkers = sorted(glob(join(matching_run,"*-pos.npy")))[::-skip]

new_walker_pool = []
for w in input_walkers:
    nw = np.load(w)
    if len(nw.shape) != 2:
        raise ValueError("Bad shape for walkers: %s" % (nw.shape,))
    nw = nw[:,ix]
    nw -= nw.mean(axis=0)[None,:]
    new_walker_pool.extend(list(nw))

if len(new_walker_pool)<walkers:
    raise ValueError("Only %d walkers found but needed %d" 
                     % (len(new_walker_pool), walkers))

random.shuffle(new_walker_pool)
new_walkers = np.array(new_walker_pool[:walkers])[None,:,:]

In [290]:
walker_name = "start-walkers.npy"
# Note shape has a 1 in front
np.save(walker_name, new_walkers)
print new_walkers.shape

(1, 98, 15)


## Manage qsub status

Check whether a job is running, start it if it isn't, inspect what went wrong if it died.

See also [Ganglia](http://localhost:9080/ganglia/?c=nimrod&m=load_one&r=hour&s=by%20name&hc=4&mc=2)

In [342]:
o = subprocess.check_output(["qstat","-f"])
jobs = {}
cj = None
ce = None
for l in o.split("\n"):
    if l.startswith("\t"):
        jobs[cj][ce] += l[1:]
    elif l.startswith(" "*4):
        k, v = l.split("=", 1)
        k = k.strip()
        v = v.strip()
        ce = k
        jobs[cj][ce] = v
    elif l.startswith("Job Id:"):
        i, j = l.split(":",1)
        cj = j.strip()
        if cj in jobs:
            raise ValueError
        jobs[cj] = {}

myjobs = {k:v for (k,v) in jobs.items() 
          if v["Job_Owner"]=="aarchiba@nimrod.cv.nrao.edu"}
for k in sorted(myjobs.keys()):
    j = myjobs[k]
    if os.path.exists(join("emcee-chains",k)):
        print k, j.get("job_state",None)

In [343]:
qsub_script = """#!/bin/sh
#PBS -lnodes={nodes}:ppn={ppn}:compute:{tag}
#PBS -N threebody_emcee_mpi
#PBS -V

cd /data2/people/aarchiba/projects/triplesystem/code-v1
export PATH=/users/aarchiba/.virtualenvs/triplesystem/bin:/users/aarchiba/.local/bin:/usr/local/src/presto/bin:/users/\
aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/b\
in:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/\
local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:.:/users/aarchiba/bin:/opt/local/share/cv/bin:/usr/local/bin:/opt/local/\
bin:/bin:/usr/bin:/bin:/usr/etc:/usr/sbin:/sbin:/usr/local/sbin:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/b\
in:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/usr/local/src/fv5.4:/usr\
/local/openmpi-2.1.0/bin/:/usr/lib64/qt-3.3/bin
export PYTHONPATH=
export OPENBLAS_NUM_THREADS=1

mpirun python emcee_chain_mpi.py

""".format(**globals())

print(qsub_script)

#!/bin/sh
#PBS -lnodes=5:ppn=20:compute:new
#PBS -N threebody_emcee_mpi
#PBS -V

cd /data2/people/aarchiba/projects/triplesystem/code-v1
export PATH=/users/aarchiba/.virtualenvs/triplesystem/bin:/users/aarchiba/.local/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/users/aarchiba/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:.:/users/aarchiba/bin:/opt/local/share/cv/bin:/usr/local/bin:/opt/local/bin:/bin:/usr/bin:/bin:/usr/etc:/usr/sbin:/sbin:/usr/local/sbin:/users/aarchiba/.local/bin:/opt/rh/python27/root/usr/bin:/opt/rh/rh-git29/root/usr/bin:/opt/rh/devtoolset-6/root/usr/bin:/usr/local/src/presto/bin:/usr/local/src/fv5.4:/usr/local/openmpi-2.1.0/bin/:/usr/lib64/qt-3.3/bin
export PYTHONPATH=
export OPENBLAS_NUM_THREADS=1

mpirun python emcee_chain_mpi.py




In [344]:
P = subprocess.Popen(["qsub","-"],stdin=subprocess.PIPE,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
out, err = P.communicate(input=qsub_script)
if P.returncode:
    raise ValueError("qsub returned %s, output:\n%s\nerror:\n%s"
                     % (P.returncode, out, err))
print out
print err

9658.nimrod.cv.nrao.edu




## Managing active jobs

In [92]:
def uniquify(l):
    s = set()
    for i in l:
        if i not in s:
            yield i
            s.add(i)


In [94]:
for k in sorted(myjobs.keys()):
    print
    print k
    j = myjobs[k]
    p = j["Output_Path"].split(":")[1]
    if os.path.exists(p):
        for l in uniquify(open(p,"rt").readlines()):
            print l,
    print
    p = j["Error_Path"].split(":")[1]
    if os.path.exists(p):
        for l in uniquify(open(p,"rt").readlines()):
            print l,
    print
    


3544.nimrod.cv.nrao.edu

/usr/local/lib/python2.7/site-packages/stevedore/extension.py:17: UserWarning: Module mpl_toolkits was already imported from None, but /home/rhlocal/rhel6/x86_64/stow/Python-2.7.3/lib/python2.7/site-packages/matplotlib-1.2.1-py2.7-linux-x86_64.egg is being added to sys.path
  import pkg_resources



In [81]:
for k in sorted(myjobs.keys()):
    print myjobs[k]["exec_host"]

nimrod06/0
